### LLM OpenAeroStruct

#### A multiagent tool to write optimization code for OpenAeroStruct with simply high level inputs from the user, i.e. I want to design a sweeped rectangular wing with elliptical lift distribution

In [1]:
# Import the generative ai library
import google.generativeai as genai #type: ignore

# Import all the modules necessary to run OpenAeroStruct
import re
import time
import os
import warnings
import numpy as np
import pandas as pd #type: ignore
import openmdao.api as om
import json

# import OpenAeroStruct modules
from openaerostruct.geometry.utils import generate_mesh  # helper functions to generate mesh
from openaerostruct.geometry.geometry_group import Geometry
from openaerostruct.aerodynamics.aero_groups import AeroPoint

# Import the plotting libraries
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import niceplots  # Optional but recommended

#Ignore warnings and use the nice plots style
warnings.filterwarnings("ignore")

plt.style.use(
    niceplots.get_style("james-dark")
)  # Options: "doumont-light", "doumont-dark", "james-light", "james-dark"

In [2]:
#Import the relevant agents
from Agents import ReformulatorAgent
from Agents import BaseMeshAgent
from Agents import GeometryAgent
from Agents import OptimizerAgent
from Agents import ResultsReaderAgent
from Agents import ReportWriter

In [3]:
# Define the initial sample query from the user.
User_Request  = """For this design, we will keep the area constant at S = 400 m2. The span is b = 60 m. The cruise condition corresponds to CL = 0.5. Your job is to minimize drag at the condition of CL = 0.5 and you have complete freedom in the taper and sweep of the wing. Make a plot of the elliptical lift distribution."""

reformulator = ReformulatorAgent()
reformulator_output = reformulator.execute_task(User_Request)

In [4]:
print("Reformulator Output:")
print(reformulator_output)

Reformulator Output:
{'objective_function': 'Minimize drag', 'trim_condition': 'CL = 0.5', 'geometric_constraint': 'Wing area (S) = 400 m^2, Span (b) = 60 m', 'design_variables': 'Taper, Sweep', 'baseline_wing_mesh': 'rect', 'optimization_algorithm': 'SLSQP', 'plotting_requirements': 'Plot of the elliptical lift distribution', 'errors': 'None'}


#### Now start writing the file into RunOAS to prepare for the optimization and plotting script using LLM.

In [5]:
#Write the mesh prompt

MeshPrompt  = f"""For this wing desgin, the geometric parameters are as follows: {reformulator_output["geometric_constraint"]}, and the type of the wing mesh should be: {reformulator_output["baseline_wing_mesh"]}"""

mesher = BaseMeshAgent()
mesher_output = mesher.execute_task(MeshPrompt)

In [6]:
print(mesher_output["python_code"])
print(mesher_output)

mesh_dict = {
    "num_y": 19, #number of panels in the y direction, 19 is a good starting number
    "num_x": 3, #number of panels in the x direction, 3 is a good starting number
    "wing_type": "rect", #This can either be "rect" or "crm" only
    "symmetry": True, # True if the wing is symmetric, False if it is not, wings are typically symmetric
    "span": 60.0, #This is the full span of the wing in meters
    "root_chord": 6.666666666666667, #This is the root chord of the wing in meters
    "span_cos_spacing": 0.0, #This is usually not edited
    "chord_cos_spacing": 0.0, #This is usually not edited
}

# Generate VLM mesh for half-wing
mesh = generate_mesh(mesh_dict)   # this creates a rectangular wing mesh, do not edit

# plot mesh
plot_mesh(mesh) # this plots the rectangular wing mesh, do not edit
{'calculations_and_explain': 'I am given the wing area (S) and span (b). I need to calculate the root chord using the formula: root_chord = area / span. In this case, root_chord = 400 

In [7]:
#Geometry Setup
GeometryPrompt  = f"""For this wing desgin, we are allowed to change the following parameters: {reformulator_output["design_variables"]}"""

geometry_setup = GeometryAgent()
geometry_output = geometry_setup.execute_task(GeometryPrompt)

In [8]:
print(geometry_output["python_code"])
print(geometry_output)

surface = {
    # Wing definition, KEEP THE SAME UNLESS ASKED TO CHANGE
    "name": "wing",  # name of the surface, keep as wing
    "symmetry": True,  # if true, model one half of wing reflected across the plane y = 0
    "S_ref_type": "wetted",  # how we compute the wing area, can be 'wetted' or 'projected'
    "mesh": mesh,

    # Aerodynamic performance of the lifting surface at an angle of attack of 0 (alpha=0).
    # These CL0 and CD0 values are added to the CL and CD obtained from aerodynamic analysis of the surface to get the total CL and CD.
    # These CL0 and CD0 values do not vary wrt alpha. DO NOT EDIT THEM UNLESS ASKED TO.
    "CL0": 0.0,  # CL of the surface at alpha=0
    "CD0": 0.0,  # CD of the surface at alpha=0

    # Airfoil properties for viscous drag calculation, DO NOT CHANGE UNLESS ASKED TO
    "k_lam": 0.05,  # percentage of chord with laminar flow, used for viscous drag
    "c_max_t": 0.303,  # chordwise location of maximum (NACA0015)
    "t_over_c_cp": np.ar

In [9]:
# Optimization Setup
OptimizerPrompt  = f"""For this wing desgin, the optimization parameters are as follows: {reformulator_output["design_variables"]}, the objective function is {reformulator_output["objective_function"]}, the geometric constraints are {reformulator_output["geometric_constraint"]}, the flight condition is {reformulator_output["trim_condition"]}, and the optimization algorithm is {reformulator_output["optimization_algorithm"]}"""

optimizer_setup = OptimizerAgent()
optimizer_output = optimizer_setup.execute_task(OptimizerPrompt)

In [10]:
print(optimizer_output["python_code"])
print(optimizer_output)

# Instantiate the problem and the model group
prob = om.Problem()

# Define flight conditions
Mach_number = 0.5 # You can change this if the user specifies a different Mach number
rho = 1.225
v = Mach_number * 340  # freestream speed, m/s
Re_c = rho * v / 1.81e-5  # Reynolds number / characteristic length, 1/m

indep_var_comp = om.IndepVarComp()
indep_var_comp.add_output("v", val=v, units="m/s")  # Freestream Velocity
indep_var_comp.add_output(
    "alpha", val=0.0, units="deg"
) 
indep_var_comp.add_output("Mach_number", val=Mach_number)  # Freestream Mach number
indep_var_comp.add_output("re", val=Re_c, units="1/m")  # Freestream Reynolds number times chord length
indep_var_comp.add_output("rho", val=rho, units="kg/m**3")  # Freestream air density
indep_var_comp.add_output("cg", val=np.zeros((3)), units="m")  # Aircraft center of gravity
prob.model.add_subsystem("flight_vars", indep_var_comp, promotes=["*"])

# Setup OpenAeroStruct model
name = surface["name"]

# Add geometry group to

### After Running the Optimization, the LLM should then have access to the plots then analyze the results

In [11]:
#Analyze the results without RAG#
#Geometry Setup
ResultsPrompt  = f"""The initial problem by the user is: {User_Request}, the reformulated problem is: {reformulator_output}"""

results_setup = ResultsReaderAgent()
results_output = results_setup.execute_task(ResultsPrompt)

In [12]:
results_output

{'Analysis': 'The optimization appears to be successful in minimizing drag, achieving a value of approximately 1.09387298 (unscaled). The lift coefficient constraint of 0.5 is met. The design variables, wing taper and sweep, converged to values of 0.2 and 30 degrees, respectively. The elliptical lift distribution plot shows a near-elliptical shape, indicating efficient drag minimization.',
 'Recommendations': 'Further optimizations could explore expanding the design space by relaxing the bounds on the taper and sweep. Alternatively, consider using different optimizers with different convergence properties to see if a lower drag can be obtained. It is important to verify the results using higher-fidelity CFD simulations as OpenAeroStruct uses a vortex lattice method which is an approximation. Also consider manufacturability.',
 'Optimization Performance': 'The optimization converged in 12 iterations, with a wall clock run time of less than a second. The optimizer used was SLSQP. Given t

In [13]:
#Analyze the results without RAG#
#Geometry Setup
ReportPrompt  = f"""The initial problem by the user is: {User_Request}, the reformulated problem is: {reformulator_output}, the analysis by the LLM is {results_output}"""

report_setup = ReportWriter()
report_output = report_setup.execute_task(ReportPrompt)

In [14]:
print(report_output["ReportText"])

\documentclass{article}
\usepackage{amsmath}
\usepackage{graphicx}

\begin{document}

\section*{Optimization Report: Wing Drag Minimization}

\subsection*{Analysis of Results}

The optimization process successfully minimized the drag, achieving a drag value of approximately 1.09387298 (unscaled). The lift coefficient constraint of $C_L = 0.5$ was accurately satisfied. The design variables converged to the following values:

\begin{itemize}
    \item Wing Taper: 0.2
    \item Wing Sweep: 30 degrees
\end{itemize}

The resulting lift distribution closely resembles an elliptical shape, which confirms the effectiveness of the drag minimization process. This near-elliptical distribution is indicative of an efficient wing design from a drag perspective.

\subsection*{Recommendations for Further Optimization}

To potentially achieve even lower drag values, the following recommendations are suggested:

\begin{enumerate}
    \item \textbf{Expand Design Space:} Relax the bounds on the taper and s

In [17]:
#Take the latex output stored as text in report_output["ReportText"] and write it into a file
with open("/Users/conan/Desktop/LLM_Aerospace_Research/LLM_OpenAeroStruct/Figures/Report.tex", "w") as f:
    f.write(report_output["ReportText"])
#Convert the latex file into a pdf using pdflatex
os.system("pdflatex Report.tex")


sh: pdflatex: command not found


32512